# Axisym PTST with unit ligament area

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np

In [2]:
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization - Force-controlled - 1N load

## Ligament of unit area

Expected $\sigma_{\mathrm{T}} = \frac{Load}{Area} = \frac{1}{1} = 1$ 

In [129]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [130]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [131]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [132]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [133]:
#bond_m.interact()

In [134]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [135]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)

In [136]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [137]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Ligament area equal to 2 - Case 1: Perimiter = 2, Height = 1

Expected $\sigma_{\mathrm{T}} = \frac{Load}{Area} = \frac{1}{2} = 0.5$ 

In [138]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 2.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [139]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [140]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [141]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [142]:
#bond_m.interact()

In [143]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [144]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)

In [145]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [146]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Ligament area equal to 2 - Case 2: Perimiter = 1, Height = 2

Expected $\sigma_{\mathrm{T}} = \frac{Load}{Area} = \frac{1}{2} = 0.5$ 

In [148]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 2.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [149]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [150]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [151]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [152]:
#bond_m.interact()

In [153]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [154]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)

In [155]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [156]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FE discretization - Displacement-controlled - 1 mm sliding

## Ligament of area = 2

Expected $\sigma_{\mathrm{T}} = E_{\mathrm{T}} *  u_{\mathrm{T}} = 1 * 1 = 1$ 
Expected $F = \sigma_{\mathrm{T}} *  Area = 1 * 2 = 2$ 

In [185]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 2.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [186]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [187]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='u', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [188]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [189]:
#bond_m.interact()

In [190]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [191]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)

In [192]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [193]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FE discretization - Checking stresses at Mayavi 

## Area of the section = 1, load = 1

Expected $\sigma_{x} =  \frac{Load}{Area} =  \frac{1}{1} = 1$ 
Expected $u_x = \frac{\sigma_{x}}{E} = \frac{1}{1} = 1$ 

In [6]:
n_y_e = 1
n_inner_x_e = 1

L_x = 1.0
R_in = 1.0

xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())

m1 = MATS3DElastic(E=1, nu=0.3)


In [18]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_support = BCSlice(slice=xd_lower.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-1.0)
# loaded_lower_dof = loaded_lower.dofs[0]
# loaded_lower_dof_2 = loaded_lower.dofs[1]

# loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -0.5)
# loaded_lower_bc_2 = BCDof(var='f', dof=loaded_lower_dof_2, value = -0.5)

# bc1 = [axisym, fixed_support, loaded_lower_bc, loaded_lower_bc_2]



In [19]:
F

0.5

In [20]:
m = TStepBC(
    domains=[(xd_lower, m1),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [21]:
#bond_m.interact()

In [22]:
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [23]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)

In [24]:
_, (ax1) = plt.subplots(1,1)
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# Plot the punch-through curve

In [14]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [15]:
xd12.o_Eia

array([[[ 2,  3],
        [ 6,  7]],

       [[ 8,  9],
        [12, 13]]])

In [16]:
m.hist.U_t[-1][xd12.o_Eia]

array([[[-1.00001650e+00,  3.58648321e-05],
        [-9.99999850e-01, -2.63154163e-05]],

       [[ 0.00000000e+00,  7.93241176e-06],
        [-1.63545591e-05, -1.82771242e-06]]])

In [17]:
xd12.map_U_to_field(U_to[-1])[0,0]

array([ 1.00001650e+00, -2.79324204e-05])

In [18]:
F_to.shape

(2, 16)

In [19]:
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [20]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [195]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [44]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [45]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [46]:
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']

In [47]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [48]:
_, (ax, ax_omega) = plt.subplots(1,2, figsize=(10,5))
idx = -1
ax_u = ax.twinx()
ax.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax_omega.plot(x_m, omega_Nx[idx,:,:].flatten(), color='green', label='omega_N')
ax_omega.plot(x_m, omega_Tx[idx,:,:].flatten(), color='green', linestyle='dashed', label='omega_T')
ax.legend()
ax_omega.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
sig_pi_Tx[-1]

array([[ 3.69767199,  3.44363753],
       [ 3.44363753,  2.75394758],
       [ 2.75394758,  1.23215998],
       [ 1.23215998, -0.55897674],
       [-0.55897674, -2.55754274],
       [-2.55754274, -3.9001756 ],
       [-3.9001756 , -5.14633512],
       [-5.14633512, -6.6572502 ]])

# FE discretization - Force-controlled

In [33]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [34]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [37]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-.5)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, -1, -1, -1],
                  var='f', dims=[0], value=-1)


In [38]:
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_dof

6

In [39]:
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1 )
unloaded_inner = BCSlice(slice=xd_lower.mesh[0, :, 0, :],
                  var='u', dims=[0], value=0)
bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [40]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [41]:
#bond_m.interact()

In [42]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



# Plot the punch-through curve

In [43]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [44]:
xd12.o_Eia

array([[[ 2,  3],
        [ 6,  7]],

       [[ 8,  9],
        [12, 13]]])

In [45]:
m.hist.U_t[-1][xd12.o_Eia]

array([[[-9.99991411e-01,  4.88573658e-06],
        [-1.00002494e+00,  4.66343961e-06]],

       [[ 0.00000000e+00,  7.93203062e-06],
        [-1.63546672e-05, -1.82725804e-06]]])

In [46]:
xd12.map_U_to_field(U_to[-1])[0,0]

array([9.99991411e-01, 3.04629405e-06])

In [47]:
F_to.shape

(2, 16)

In [48]:
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [49]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [50]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [51]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [52]:
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']

In [53]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [54]:
_, (ax, ax_omega) = plt.subplots(1,2, figsize=(10,5))
idx = -1
ax_u = ax.twinx()
ax.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax_omega.plot(x_m, omega_Nx[idx,:,:].flatten(), color='green', label='omega_N')
ax_omega.plot(x_m, omega_Tx[idx,:,:].flatten(), color='green', linestyle='dashed', label='omega_T')
ax.legend()
ax_omega.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
sig_pi_Tx[-1]

array([[ 3.69767199,  3.44363753],
       [ 3.44363753,  2.75394758],
       [ 2.75394758,  1.23215998],
       [ 1.23215998, -0.55897674],
       [-0.55897674, -2.55754274],
       [-2.55754274, -3.9001756 ],
       [-3.9001756 , -5.14633512],
       [-5.14633512, -6.6572502 ]])